In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet
import random
import warnings
warnings.filterwarnings('ignore')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
set_seed(12)


/opt/miniconda3/envs/openfl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fx.init('torch_cnn_mnist', log_level='METRIC', log_file='./spam_metric.log')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── data
│   └── MNIST
│       └── raw
├── src
│   ├── __pycache__
│   │   ├── mnist_utils.cpython-39.pyc
│   │   └── __init__.cpython-39.pyc
│   ├── pt_cnn.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── ptmnist_inmemory.py
├── cert
├── logs
│   └── cnn_mnist
│       ├── events.out.tfevents.1682333896.claret.216878.0
│       ├── events.out.tfevents.1682320032.claret.166375.0
│       ├── events.out.tfevents.1682323085.claret.169851.0
│       ├── events.out.tfevents.1682341503.claret.226136.0
│       ├── events.out.tfevents.1682323189.claret.170265.0
│       ├── events.out.tfevents.1681901929.claret.23011.0
│       ├── events.out.tfevents.1682332450.claret.214299.0
│       ├── events.out.tfevents.1682195294.claret.101800.1
│       ├── events.out.tfevents.1682333537.claret.216050.0
│       ├── events.out.tfevents.1682323291.claret.170452.0
│       ├── events.out.tfevents.168232

In [3]:
# TODO load data
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_data,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_data = torch.from_numpy(np.expand_dims(train_data, axis=1)).float()
train_labels = one_hot(train_labels,10)

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_data,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_data = torch.from_numpy(np.expand_dims(valid_data, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

In [4]:
from openfl.utilities.optimizers.torch import FedProxOptimizer

In [5]:
i = 'ckpt4'

feature_shape = train_data.shape[1]
classes       = 10

for j in range(10):
    fl_data = FederatedDataSet(train_data,train_labels,valid_data,valid_labels,batch_size=32,num_classes=classes)


    class Model(nn.Module):
        def __init__(self):
            super(Model, self).__init__()
            if 'model' in vars() or 'model' in globals():
                self.model = model.model
            else:
                self.model = nn.Sequential(
                    nn.Flatten(),
                    nn.Linear(28*28, 128),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(128, 64),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(64, 10),
                )

        def forward(self, x):
            return self.model(x)

        def train_epoch(self, batch_generator):
            from openfl.federated.task import PyTorchTaskRunner
            self.optimizer.set_old_weights([p for p in self.parameters()])
            return PyTorchTaskRunner.train_epoch(self, batch_generator)

    optimizer = lambda x: FedProxOptimizer(x, lr=1e-4, mu=0.1)

    def cross_entropy(output, target):
        """Binary cross-entropy metric
        """
        return F.binary_cross_entropy_with_logits(input=output,target=target.float())

    fl_model = FederatedModel(build_model=Model, optimizer=optimizer, loss_fn=cross_entropy, data_loader=fl_data)

    collaborator_models = fl_model.setup(num_collaborators=4)
    collaborators = {'one':collaborator_models[0],'two':collaborator_models[1], 'three':collaborator_models[2] , 'four':collaborator_models[3]}
    final_fl_model = fx.run_experiment(
        collaborators,
        {
            'aggregator.settings.rounds_to_train': 1,
            'collaborator.settings.opt_treatment': 'CONTINUE_GLOBAL',
        }
    )
    torch.save(final_fl_model.model, f'/home/p/Desktop/Programs/Python/openfl/openfl-test/test/final_model_{i}.{j}.pt')
    model = final_fl_model.model.cpu()

[18:56:40] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=149527;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py\utils.py]8;;\:]8;id=400179;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py#172\172]8;;\

[18:56:41] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=674662;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py\utils.py]8;;\:]8;id=849711;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=238702;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py\utils.py]8;;\:]8;id=585304;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=461430;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py\utils.py]8;;\:]8;id=385514;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=61681;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py\utils.py]8;;\:]8;id=604204;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/utilities/utils.py#172\172]8;;\

...

           METRIC   Round 0, collaborator validate_local locally_tuned_model_validation result acc: 0.753200                                   ]8;id=376209;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=325287;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py#559\559]8;;\

           METRIC   Round 0, aggregator: locally_tuned_model_validation                                                                        ]8;id=928862;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=54118;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    <openfl.interface.aggregation_functions.weighted_average.WeightedAverage object at 0x7f2dc10bf580> acc:     0.756500                        

           METRIC   Round 0, aggregator: train <openfl.interface.aggregation_functions.weighted_average.WeightedAverage object at              ]8;id=588404;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=739212;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    0x7f2dc10bf580> cross_entropy:    0.213156                                                                                                  

           METRIC   Round 0, aggregator: aggregated_model_validation <openfl.interface.aggregation_functions.weighted_average.WeightedAverage  ]8;id=92406;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=420613;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    object at 0x7f2dc10bf580> acc:        0.742800                                                                                              

           METRIC   Round 0: saved the best model with score 0.742800                                                                          ]8;id=363292;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=105742;file:///home/p/Desktop/Programs/Python/openfl/openfl/openfl/component/aggregator/aggregator.py#858\858]8;;\

In [6]:
#Original MNIST dataset
print(f'Original training data size: {len(train_data)}')
print(f'Original validation data size: {len(valid_data)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')


Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 15000
Collaborator one's validation data size: 2500

Collaborator two's training data size: 15000
Collaborator two's validation data size: 2500



In [7]:
 #Get the current values of the plan. Each of these can be overridden
import json
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 2,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.log_metric_callback.template": "src.mnist_utils.write_metric",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.settings.write_logs": true,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups.0.name": "train_and_validate",
    "assigner.settings.task_groups.0.percentage": 1.0,
    "assigner.settings.task_groups.0.tasks.0": "aggregated_model_validation",
    "assigner.settings.task_groups.0.tasks.1": "train",
    "assigner.settings.task_groups.0.tasks.2": "locally_tuned_model_validation",
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_

In [8]:
#Run experiment, return trained FederatedModel


In [9]:
#Save final model


In [10]:
(final_fl_model.model.cpu()(valid_data).argmax(dim=-1) == torch.tensor(valid_labels).argmax(dim=-1)).sum() / len(valid_labels)

tensor(0.7576)

In [11]:
FederatedModel?